<a href="https://colab.research.google.com/github/FPX-Rain/-/blob/master/kaggle_train_great_reff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"darkrain","key":"198c80bf7c47f0fad6c6e865d102b532"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c tensorflow-great-barrier-reef -p '/content/sample_data/kaggle'

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 2.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=df5df128262646af7efc1dad2e83d7809967fb6fa15accfc8b32bd3198d1edfe
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir -p ~/.kaggle
!echo '{"username":"darkrain","key":"198c80bf7c47f0fad6c6e865d102b532"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c tensorflow-great-barrier-reef -p '/content/sample_data/kaggle'

100% 14.2G/14.2G [05:37<00:00, 74.4MB/s]
100% 14.2G/14.2G [05:37<00:00, 45.1MB/s]


In [ ]:
%cd /content/sample_data/kaggle

/content/sample_data/kaggle


In [ ]:
!ls

In [ ]:
!unzip tensorflow-great-barrier-reef.zip

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [ ]:
train = pd.read_csv('/content/sample_data/kaggle/train.csv')
train['pos'] = train.annotations != '[]'

In [ ]:
!mkdir -p /content/drive/MyDrive/kaggle_data/yolo_data/fold1/images/val
!mkdir -p /content/drive/MyDrive/kaggle_data/yolo_data/fold1/images/train

!mkdir -p /content/drive/MyDrive/kaggle_data/yolo_data/fold1/labels/val
!mkdir -p /content/drive/MyDrive/kaggle_data/yolo_data/fold1/labels/train

In [ ]:
fold = 1

annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'/content/sample_data/kaggle/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'/content/drive/MyDrive/kaggle_data/yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'/content/drive/MyDrive/kaggle_data/yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [ ]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [ ]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/fold1/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git '/content/drive/MyDrive/yolo'

Cloning into '/content/drive/MyDrive/yolo'...
remote: Enumerating objects: 10628, done.
remote: Total 10628 (delta 0), reused 0 (delta 0), pack-reused 10628
Receiving objects: 100% (10628/10628), 10.84 MiB | 7.36 MiB/s, done.
Resolving deltas: 100% (7335/7335), done.


In [ ]:
import torch

In [ ]:
with open('/content/drive/MyDrive/yolo/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('/content/drive/MyDrive/yolo/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [ ]:
 %cd /content/drive/MyDrive/yolo

/content/drive/MyDrive/yolo


In [ ]:
!python train.py --img 2400 --batch 1 --epochs 5 --data reef_f1_naive.yaml --name rain_kaggle_model_1 --hyp data/hyps/hyp.heavy.2.yaml --optimizer Adam --weight yolov5m.pt

train: weights=yolov5m.pt, cfg=, data=reef_f1_naive.yaml, hyp=data/hyps/hyp.heavy.2.yaml, epochs=5, batch_size=1, imgsz=2400, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=runs/train, name=rain_kaggle_model_1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 6 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-207-g8efe977 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_

In [ ]:
!pip install -r requirements.txt


In [ ]:
%cd models

In [ ]:
!python yolo.py --cfg yolo_rain.yaml